<a href="https://colab.research.google.com/github/pierfrancescomartinello/NLP-Project/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install farm-haystack[colab,faiss]==1.17.2

## Setup

In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [165]:
import pandas as pd
import numpy as np

filepath = "/content/drive/MyDrive/output240617110842.json"
df = pd.read_json(filepath)
df.columns = ["title", "addr", "text"]

df

,title,addr,text
0,Università degli Studi di Palermo,https://www.unipa.it/,
1,Organi di Governo | Università degli Studi di ...,https://www.unipa.it/ateneo/OrganiDiGovernoECo...,
2,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
3,Presentazione | Università degli Studi di Palermo,https://www.unipa.it/ateneo/presentazione/,
4,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
...,...,...,...
6333,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Ca...
6334,| Università degli Studi di Palermo,https://www.unipa.it/amministrazione/rettorato...,
6335,Settore Comunicazione e URP | Settore Comunica...,https://www.unipa.it/amministrazione/rettorato...,
6336,Regolamenti per aree tematiche di interesse | ...,https://www.unipa.it/servizi/prevenzionedellac...,REGOLAMENTI PERSONALE DOCENTE E RICERCATORE


## Preprocess

In [166]:
import re

def remove_dates(s: str) -> str:
    regex = r'\d{1,2}-(gen|feb|mar|apr|mag|giu|lug|ago|set|ott|nov|dic)-\d{4}'

    return re.sub(regex, '', s)

def remove_empty_docs(df):
    df["text"] = df["text"].replace("", np.nan)
    df.dropna(subset=["text"], inplace=True)

    return df

def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df["text"] = df["text"].apply(lambda x: x.strip())
    df = remove_empty_docs(df)
    df["text"] = df["text"].apply(remove_dates)

    # specify minimum document length
    df = df[df["text"].str.len() >= 200]

    df.reset_index(inplace=True, drop=True)

    return df

In [167]:
tmp = preprocess(df)
tmp

,title,addr,text
0,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
1,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
2,Sostegno allo studio | Centro per l’innovazion...,https://www.unipa.it/strutture/cimdu/Sostegno-...,"Da 15 anni ItaStra, Scuola di Lingua Italian..."
3,PNRR | PNRR | Università degli Studi di Palermo,https://www.unipa.it/progetti/pnrr/,Il Piano Nazionale di Ripresa e Resilienza - P...
4,Corsi di preparazione alle prove di accesso A....,https://www.unipa.it/Corsi-di-preparazione-all...,Sono aperte le iscrizioni all’edizione inverna...
...,...,...,...
2774,U.O. Didattica e Internazionalizzazione - Desc...,https://www.unipa.it/strutture/cimdu/U.O.-Dida...,Il Responsabile dell’U.O. Didattica e Interna...
2775,Ricercatori neoassunti | Università degli Stud...,https://www.unipa.it/strutture/cimdu/ricercato...,Per restare sempre aggiornati è possiblie iscr...
2776,Declaratorie della U.O. | Centro per l’innovaz...,https://www.unipa.it/strutture/cimdu/Declarato...,La U.O. Didattica e Internazionalizzazione si...
2777,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Cal...


## RAG setup

In [14]:
from haystack import Document

# Use data to initialize Document objects
titles = list(df["title"].values)
texts = list(df["text"].values)
documents = []
for title, text in zip(titles, texts):
    documents.append(Document(content=text, meta={"name": title or ""}))


In [17]:
from haystack.document_stores import FAISSDocumentStore
import os

# force store re-creation
store_path = "/content/faiss_document_store.db"
if os.path.exists(store_path):
    os.remove(store_path)

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)


In [16]:
from haystack.nodes import RAGenerator, DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


KeyboardInterrupt: 

In [ ]:
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=200,
    min_length=2,
    embed_title=True,
    num_beams=3,
)


In [ ]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)


In [ ]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)


### Machine Translation

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

mt_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-it-en", framework="pt", max_length=70000,)

## QA

In [ ]:
from haystack.utils import print_answers

QUESTIONS = [
    # "chi ha fondato Unipa?",
    # "who founded Unipa?",
    # "quando è stata fondata Unipa?",
    # "when was Unipa founded?",
    # "cos'è Data Algorithms and Machine Intelligence",
    # "what is è Data Algorithms and Machine Intelligence",
    # "cos'è il phishing?",
    # "cosa raccoglie la pagina?",
    "a cosa serve la carta europea?",
    "stazione",
    "biblioteche universitarie",
    "corsi di laurea di unipa"

]

for question in QUESTIONS:
    res = pipe.run(query=question, params={"Generator": {"top_k": 3}, "Retriever": {"top_k": 10}})
    print_answers(res, details="medium")
    print()
